**🛠️ Steps to Train U-Net for Pool Segmentation**

1️⃣ Prepare Data (Images & Masks)

2️⃣ Preprocess Data (Resize, Normalize, Augment)

3️⃣ Define U-Net Model

4️⃣ Train U-Net Model

5️⃣ Evaluate & Test

##  Step 2: Data Preprocessing

In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [3]:
# Paths
IMAGE_DIR = "../dataset_portion/images/"
MASK_DIR = "../dataset_portion/masks/"

# Load images and masks
image_files = sorted(os.listdir(IMAGE_DIR))
mask_files = sorted(os.listdir(MASK_DIR))

# Image size for U-Net (resize all images)
IMG_SIZE = 512

In [4]:
def preprocess_image(img_path):
    img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))  # Resize
    img = img_to_array(img) / 255.0  # Normalize (0-1)
    return img

def preprocess_mask(mask_path):
    mask = load_img(mask_path, target_size=(IMG_SIZE, IMG_SIZE), color_mode="grayscale")
    mask = img_to_array(mask) / 255.0  # Normalize (0-1)
    mask = np.where(mask > 0.5, 1, 0)  # Binarize (0 or 1)
    return mask

# Load dataset
X = np.array([preprocess_image(os.path.join(IMAGE_DIR, f)) for f in image_files])
Y = np.array([preprocess_mask(os.path.join(MASK_DIR, f)) for f in mask_files])

# Split into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

print("✅ Data Loaded! Training images:", X_train.shape, "Training masks:", Y_train.shape)


✅ Data Loaded! Training images: (52, 512, 512, 3) Training masks: (52, 512, 512, 1)


## Step 3: Define U-Net Model

In [5]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Input
from tensorflow.keras.models import Model

In [6]:
def unet_model(input_size=(IMG_SIZE, IMG_SIZE, 3)):
    inputs = Input(input_size)

    # Encoder (Downsampling)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    # Decoder (Upsampling)
    u5 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = concatenate([u5, c3])
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c2])
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c1])
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c7)  # Binary mask output

    model = Model(inputs, outputs)
    return model

# Compile model
model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("✅ U-Net Model Ready!")


✅ U-Net Model Ready!


##  Step 4: Train the Model

In [ ]:
# Train the model
history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=11, #30
    batch_size=4, #8
    verbose=1
)


Epoch 1/11
 2/13 [===>..........................] - ETA: 16:41 - loss: 0.6123 - accuracy: 0.9892

## Step 5: Evaluate & Test

In [ ]:
# Test on a sample image
test_idx = 0
test_image = X_val[test_idx]
test_mask = Y_val[test_idx]

In [ ]:
# Predict mask
pred_mask = model.predict(np.expand_dims(test_image, axis=0))[0]
pred_mask = (pred_mask > 0.5).astype(np.uint8)  # Binarize output

In [ ]:
# Display results
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(test_image)
plt.title("Original Image")

plt.subplot(1, 3, 2)
plt.imshow(test_mask, cmap="gray")
plt.title("Ground Truth Mask")

plt.subplot(1, 3, 3)
plt.imshow(pred_mask, cmap="gray")
plt.title("Predicted Mask")

plt.show()

In [ ]:
# Save the model in HDF5 format
model.save("unet_model.h5")
print("✅ Model saved as 'unet_model.h5'!")

# **🛠️ Steps to Overlay Contour on Image**
1️⃣ Convert predicted mask into a binary image

2️⃣ Use cv2.findContours to detect edges

3️⃣ Draw contours using cv2.drawContours on the original image

4️⃣ Display the final result

🔹  Draw Contour on Image


In [ ]:
# Find contours in the predicted mask
contours, _ = cv2.findContours(pred_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours on the original image (green color, thickness 2)
contour_image = test_image.copy()
cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 1)  # Green contours

# Display Results
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(test_image)
plt.title("Original Image")

plt.subplot(1, 3, 2)
plt.imshow(pred_mask, cmap="gray")
plt.title("Predicted Mask")

plt.subplot(1, 3, 3)
plt.imshow(contour_image)
plt.title("Contour Overlay")

plt.show()
